## SimpleBaseline

### STEP 1 : simplebaseline 모델 완성하기

simplebaseline.py 파일 내용을 완성합니다.

```python
import tensorflow as tf

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())

    return seq_model

upconv = _make_deconv_layer(3)
final_layer = tf.keras.layers.Conv2D(16, kernel_size=(1,1), padding='same')

def Simplebaseline(input_shape=(256, 256, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    
    x = resnet(inputs)
    x = upconv(x)
    out = final_layer(x)

    model = tf.keras.Model(inputs, out, name='simple_baseline')
    return model
```

### STEP 2 : simplebaseline 모델로 변경하여 훈련하기


train.py 218라인의 모델 선언 부분을 simplebaseline 모델로 변경한 후 다시 학습을 진행합니다.



```python

# ...생략... train.py 참고

def train(epochs, start_epoch, learning_rate, tensorboard_dir, checkpoint,
          num_heatmap, batch_size, train_tfrecords, val_tfrecords, version, flag):
    strategy = tf.distribute.MirroredStrategy()
    global_batch_size = strategy.num_replicas_in_sync * batch_size
    train_dataset = create_dataset(
        train_tfrecords, global_batch_size, num_heatmap, is_train=True)
    val_dataset = create_dataset(
        val_tfrecords, global_batch_size, num_heatmap, is_train=False)

    if not os.path.exists('./models'):
        os.makedirs('./models/')

    with strategy.scope():
        train_dist_dataset = strategy.experimental_distribute_dataset(
            train_dataset)
        val_dist_dataset = strategy.experimental_distribute_dataset(
            val_dataset)
        
        if flag:
            model = Simplebaseline(IMAGE_SHAPE) # IMAGE_SHAPE 인자 전달
        else:
            model = StackedHourglassNetwork(IMAGE_SHAPE, 4, 1, num_heatmap)
        
        if checkpoint and os.path.exists(checkpoint):
            model.load_weights(checkpoint)

        trainer = Trainer(
            model,
            epochs,
            global_batch_size,
            strategy,
            initial_learning_rate=learning_rate,
            start_epoch=start_epoch,
            version=version,
            tensorboard_dir=tensorboard_dir)

        print('Start training...')
        return trainer.run(train_dist_dataset, val_dist_dataset)

# 생략...

```

### STEP 3 : 두 모델의 비교

실습에서 다룬 StackedHourglass Network와 Simplebaseline 모델을 둘 다 동일한 Epoch 수만큼 학습하여 그 결과를 비교해 봅니다.

- Pose Estimation 결과 시각화 (정성적 비교)
- 학습 진행경과 (loss 감소현황)   

가급적 두 모델 공히 최소 3epoch이상, (5epoch 이상 권장)을 학습하기 바랍니다.

두 모델의 Loss 감소 현황을 비교하기 위해 train.py의 run 메소드를 일부 수정

```python
def run(self, train_dist_dataset, val_dist_dataset):
    @tf.function
    def distributed_train_epoch(dataset):
        tf.print('Start distributed traininng...')
        total_loss = 0.0
        num_train_batches = 0.0
        for one_batch in dataset:
            per_replica_loss = self.strategy.experimental_run_v2(
                self.train_step, args=(one_batch, ))
            batch_loss = self.strategy.reduce(
                tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None)
            total_loss += batch_loss
            num_train_batches += 1
            tf.print('Trained batch', num_train_batches, 'batch loss',
                     batch_loss, 'epoch total loss', total_loss / num_train_batches)
        return total_loss, num_train_batches

    @tf.function
    def distributed_val_epoch(dataset):
        total_loss = 0.0
        num_val_batches = 0.0
        for one_batch in dataset:
            per_replica_loss = self.strategy.experimental_run_v2(
                self.val_step, args=(one_batch, ))
            num_val_batches += 1
            batch_loss = self.strategy.reduce(
                tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None)
            tf.print('Validated batch', num_val_batches, 'batch loss',
                     batch_loss)
            if not tf.math.is_nan(batch_loss):
                # TODO: Find out why the last validation batch loss become NaN
                total_loss += batch_loss
            else:
                num_val_batches -= 1

        return total_loss, num_val_batches

    summary_writer = tf.summary.create_file_writer(self.tensorboard_dir)
    summary_writer.set_as_default()

    history = {'train_loss':[], 'val_loss':[]} # 두 모델의 정량적 평가를 위해 추가
    for epoch in range(self.start_epoch, self.epochs + 1):
        tf.summary.experimental.set_step(epoch)

        self.lr_decay()
        tf.summary.scalar('epoch learning rate',
                          self.current_learning_rate)

        print('Start epoch {} with learning rate {}'.format(
            epoch, self.current_learning_rate))

        train_total_loss, num_train_batches = distributed_train_epoch(
            train_dist_dataset)
        train_loss = train_total_loss / num_train_batches
        history['train_loss'].append(train_loss) # 히스토리
        print('Epoch {} train loss {}'.format(epoch, train_loss))
        tf.summary.scalar('epoch train loss', train_loss)

        val_total_loss, num_val_batches = distributed_val_epoch(
            val_dist_dataset)
        val_loss = val_total_loss / num_val_batches
        history['train_loss'].append(val_loss) # 히스토리
        print('Epoch {} val loss {}'.format(epoch, val_loss))
        tf.summary.scalar('epoch val loss', val_loss)

        # save model when reach a new lowest validation loss
        if val_loss < self.lowest_val_loss:
            self.save_model(epoch, val_loss)
            self.lowest_val_loss = val_loss
        self.last_val_loss = val_loss

    return self.best_model, history
```

![](./1.png)

![](./2.png)

### hourglass vs simplebaseline

이미지 결과로는 두 모델의 우열을 가리기 어렵다.   
다만 골반쪽만 보면 hourglass가 조금 더 나아보인다.

학습 경과로는 simplebaseline이 처음부터 낮은 Loss값으로 시작하고 변화폭이 상대적으로 작고, hourglass가 상대적으로 높은 Loss값으로 시작해 변화폭이 조금 더 크다.   
다만 변화폭은 크게 차이나는 편은 아니다.

### 회고

- 파이썬 파일들을 적절히 수정하여 노드를 진행했다.   
- 새롭게 작성해야 하는 부분은 없어서 진행 자체는 수월했지만, 학습에 걸리는 시간이 심각하게 오래 걸린다. 
- 둘 다 스크린샷을 찍어서 조금 차이가 있지만 비슷한 크기인데 왜 주피터 노트북 마크다운 문법을 이용하면 저렇게 다르게 나올까